<div style="font-family: 'Gen Jyuu Gothic Monospace Medium', 'Noto Sans TC'; font-size: 115%; line-height: 135%; text-align: center; color: Plum;">

<br>

# String Interning 字串駐留          

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 115%; line-height: 135%;">

* 本文附帶說明字串為何是immutable(不可變)的。
* 原因先賣個關子，稍後各位自然了解...

In [35]:
import sys
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
print("InteractiveShell set.")
print(sys.version)

print()
print()
print('eeeeeee')

InteractiveShell set.
3.10.6 (main, Nov  2 2022, 18:53:38) [GCC 11.3.0]


<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 115%; line-height: 135%;">

### 設定兩兩內容相同的物件
* 目的是觀察：
    * 內容是否相等，用`==`。
    * 是否同一物件，用`is`。

In [3]:
# def compare_objs(obj1: object, obj2: object, var1: str, var2: str) -> None:
def compare_objs(*obj) -> None:
    # this_type = str(type(obj[0])).split()[1][1:-2]
    equal_operator = f'({obj[2]} == {obj[3]}) = {obj[0] == obj[1]}'
    is_operator = f'({obj[2]} is {obj[3]}) = {obj[0] is obj[1]}'
    print(f'{equal_operator:30}{is_operator}')
    # print(obj1)

print('compare_objs() defined.')    

compare_objs() defined.


In [58]:
str1 = 'Alex';        str2 = 'Alex'
int1 = 23;            int2 = 23
float1 = 0.0;         float2 = 0.0
bool1 = True;         bool2 = True
none1 = None;         none2 = None
list1 = [23];         list2 = [23]
tuple1 = (23,);       tuple2 = (23,)
set1 = {33, 23};      set2 = {33, 23}
dict1 = {'id': 23};   dict2 = {'id': 23}
objs = ((str1, str2, 'str1', 'str2'), (int1, int2, 'int1', 'int2'), (float1, float2, 'float1', 'float2'), (bool1, bool2, 'bool1', 'bool2'), (none1, none2, 'none1', 'none2'), 
        (list1, list2, 'list1', 'list2'), (tuple1, tuple2, 'tuple1', 'tuple2'), (set1, set2, 'set1', 'set2'), (dict1, dict2, 'dict1', 'dict2'))

# print('\n'*3)
for obj in objs:
    compare_objs(*obj)

(str1 == str2) = True         (str1 is str2) = False
(int1 == int2) = True         (int1 is int2) = True
(float1 == float2) = True     (float1 is float2) = False
(bool1 == bool2) = True       (bool1 is bool2) = True
(none1 == none2) = True       (none1 is none2) = True
(list1 == list2) = True       (list1 is list2) = False
(tuple1 == tuple2) = True     (tuple1 is tuple2) = False
(set1 == set2) = True         (set1 is set2) = False
(dict1 == dict2) = True       (dict1 is dict2) = False


In [21]:
# 上cell的code等於這樣一個個寫。
obj1 = 'Alex'; obj2 = 'Alex'
compare_objs(obj1, obj2, 'obj1', 'obj2')

obj1 = 23; obj2 = 23
compare_objs(obj1, obj2, 'obj1', 'obj2')

...

(obj1 == obj2) = True         (obj1 is obj2) = True
(obj1 == obj2) = True         (obj1 is obj2) = True


Ellipsis

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 115%; line-height: 135%;">

### 先聚焦在字串。

* 以`is`判斷str1和str2，結果傳回`True`，表示str1和str2是同一個物件。
    ![](./assets/string%20interning-6.png)
* 以下證明兩個字串變數，其值相同時是同一物件，看id()就知道了。

In [28]:
name1 = 'Alex'
name2 = 'Alex'
print(f'{name1 = }\t\t{name2 = }')
print(f'{id(name1) = :<25}{id(name2) = }')
print(f'{(id(name1) == id(name2)) = }')

name1 = 'Alex'		name2 = 'Alex'
id(name1) = 140047414407408          id(name2) = 140047414407408
(id(name1) == id(name2)) = True


<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 115%; line-height: 135%;">

### 為甚麼是同一物件？

* 一些現代程式語言如Java, C#, Python, PHP, Ruby, Julia, ...等，如果多個字串變數(variables)內容相同，只會留下一份copy，這些變數全指向該copy的reference。意思是多個變數共用同一份字串，而非分別擁有獨立字串。
* 這個只留一份字串的機制，稱為`string interning`，intern有扣留、拘禁之意。
* `string interning`的目的，是提升時間和空間使用的效能。可節省字串的儲存記憶體空間，在比對時更有效率：
    * 節省空間不言而喻，一份copy當然比多份copies少占用記憶體。
    * 比對時更有效率這事也不難理解，只比對一個reference(記憶體位置)，肯定快於逐個字元取出比對。

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 115%; line-height: 135%;">
  
### 番外篇：隨著`string interning`而來的限制

* 正因為有interning機制，string就必須是immutable，即其內容不可改變。
* 假如程式語言有`string interning`機制，但字串卻設計成mutable(內容可變)，則修改某一變數內容，其他interning的字串都會隨之變更。這個「連動效果」通常不是我們想要的，我們希望的是每個變數相互獨立而非相互牽扯。

In [57]:
his_name = 'Liam'             # '': ASCII 127
her_name = 'Liam'

print(f'{his_name = }\t\t\t{her_name = }')
print(f'{id(his_name) = :<25}{id(her_name) = }')
print(f'{(id(his_name) == id(her_name)) = }')

# his_name[-1] = 'n'    # 假如允許這樣修改his_name...
# print(f'{his_name = }\t\t\t{her_name = }')   # 那麼her_name也會跟著更改。
# his_name = (his_name[:-1] + 'n')    # 真要改要這樣，不過這是另起一個新字串。

his_name = 'Liam'			her_name = 'Liam'
id(his_name) = 140047410843568          id(her_name) = 140047410843568
(id(his_name) == id(her_name)) = True


<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 115%; line-height: 135%;">

### 回主線：「金無足赤，人無完人。」

「夫尺有所短，寸有所長，物有所不足，智有所不明，數有所不逮，神有所不通。」--屈原《楚辭．卜居》

---
* 天下沒有白吃的午餐，也沒有完美無缺的機制。
* `string interning`固然可提高效能和節約空間，不過Interning字串的過程，以及維護interned字串的pool，也是需要時間的。
* 如果某個字串很長很長，又不會(或絕少)和其他字串比較，也許就不值得花額外時間去intern這個字串了。
* 所以，為了平衡帶來的紅利和付出的代價，Python並不會intern所有字串，而是訂出些「遊戲規則」來判斷某字串要不要intern。
* 麻煩來了：這些遊戲規則，會讓`string interning`機制「天威難測」。原因是規則本身就繁瑣，再加上不同的Python實作(註1)可能有不同規則，即使是同一實作(如最普及的CPython)，也可能因不同版本(註2)而規則有異。
---
* 註1：如CPython, IronPython, Jython, PyPy, MicroPython, ...
* 註2：例如CPython 3.7版以前使用[peephole optimization](https://github.com/python/cpython/blob/0f21fe6155227d11dc02bd3ef3b061de4ecea445/Python/peephole.c)方法決定是否intern字串，而3.8版以後則改為[AST optimizer](https://github.com/python/cpython/blob/3.7/Python/ast_opt.c)。本文在CPython 3.10.6版測試。

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 115%; line-height: 135%;">

## 一些規則例子，肯定不周延也無意周延

### CPython 3.10版`string interning`規則-1


* 字串中有空白者不intern。

In [6]:
name1 = 'Alex Van'
name2 = 'Alex Van'

print(f'{name1 = }\t\t\t{name2 = }')
print(f'{id(name1) = :<28}{id(name2) = }')
print(f'{(id(name1) == id(name2)) = }')

name1 = 'Alex Van'			name2 = 'Alex Van'
id(name1) = 139815190966896             id(name2) = 139815190966384
(id(name1) == id(name2)) = False


![](./assets/string%20interning-8.png)

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 115%; line-height: 135%;">

### CPython 3.10版`string interning`規則-2

* 全部由ASCII 127以內字元組成的字串才會intern。
* 所以包含中文的字串是不會有`string interning`的。

In [42]:
os1 = 'Linux©'
os2 = 'Linux©'

print(f'{os1 = }\t\t\t{os2 = }')
print(f'{id(os1) = :<22}{id(os2) = }')
print(f'{(id(os1) == id(os2)) = }')

os1 = 'Linux©'			os2 = 'Linux©'
id(os1) = 140576191970432       id(os2) = 140576192144976
(id(os1) == id(os2)) = False


![](./assets/string%20interning-12.png)
![](./assets/string%20interning-13.png)

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 115%; line-height: 135%;">

* 所以包含中文的字串是不會有`string interning`的。

In [49]:
name1 = '張曼玉'
name2 = '張曼玉'

print(f'{name1 = }\t\t{name2 = }')
print(f'{id(name1) = :<20}{id(name2) = }')
print(f'{(id(name1) == id(name2)) = }')

name1 = '張曼玉'		name2 = '張曼玉'
id(name1) = 140576192098544     id(name2) = 140576192098064
(id(name1) == id(name2)) = False


![](./assets/string%20interning-14.png)

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 115%; line-height: 135%;">

### CPython 3.10版`string interning`規則-3

* 長度超過4,096個字元的字串不intern。
* CPython 3.7版本以前長度限制為20個字元。

In [8]:
str1 = 'A' * 4_096
str2 = 'A' * 4_096
str3 = 'A' * 4_096
str4 = 'A' * 4_096
str5 = 'A' * 4_096
print(f'{id(str1)=:<20}{id(str2)=:<20}{id(str3)=:<20}{id(str4)=:<20}{id(str5)=}')
print(f'{(id(str1) == id(str2) == id(str3) == id(str4) == id(str5)) = }')

id(str1)=94237235989152      id(str2)=94237235989152      id(str3)=94237235989152      id(str4)=94237235989152      id(str5)=94237235989152
(id(str1) == id(str2) == id(str3) == id(str4) == id(str5)) = True


![](./assets/string%20interning-9.png)
![](./assets/string%20interning-10.png)

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 115%; line-height: 135%;">

### CPython 3.10版`string interning`規則-4

* 字串究竟是由string variable或string literal(註1)構成，規則大致如下列(有例外)：
    * 下面的字串「會」觸發`string interning`:
        * 單一variable
        * 單一literal
        * literal + literal
    * 下面的字串「不會」觸發`string interning`:
        * variable + variable
        * variable + literal
        * literal + variable
* 夠繁瑣了吧？
---
* 註1：variable是「變數」，literal為「定數」。例如在 <span style="font-size: 135%; font-weight: 800">`name = 'Alex'`</span> 中，`name`是variable，`'Alex'`則是literal。

In [35]:
first = 'first'
last = 'last'
name1 = 'first' + 'last'
name2 = 'first' + 'last'

print(f'{name1 = }\t\t{name2 = }')
print(f'{id(name1) = :<20}{id(name2) = }')
print(f'{(id(name1) == id(name2)) = }')

name1 = 'firstlast'		name2 = 'firstlast'
id(name1) = 140576191864048     id(name2) = 140576191852720
(id(name1) == id(name2)) = False


<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 115%; line-height: 135%;">

### Life is Short

* 人生苦短，為何要花寶貴精力去記憶這些瑣碎的規則？
* 況且規則還可能改來改去。
* 時間應該用於有生產力的專案本身，而不應強記繁複又易變動的內部規則。
    ![](./assets/time%20and%20money.jpg)

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 115%; line-height: 135%;">

### 有簡單的解決辦法嗎？

* 請放心，Python永遠都會幫我們開扇窗的。
    ![](./assets/open%20window-1.jpg)

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 115%; line-height: 135%;">

### 內建的intern()函數

* 上面所述的是Python自動提供的內隱式(implicit)`string interning`機制。
* 但此機制太過冗雜，建議不用記，也不要倚賴它來判斷字串是否為同一物件。
* 如果想有interning效果，請用Python內建的intern()函數外顯設定(intern a string explicitly)。

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 115%; line-height: 135%;">

![](./assets/Python%20the%20snake-1.jpg)

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 115%; line-height: 135%;">

### 原規則：字串中有空白者不intern

In [9]:
name1 = 'Alex Van'
name2 = 'Alex Van'

print(f'{name1 = }\t\t\t{name2 = }')
print(f'{id(name1) = :<28}{id(name2) = }')
print(f'{(id(name1) == id(name2)) = }')

name1 = 'Alex Van'			name2 = 'Alex Van'
id(name1) = 140618241819824             id(name2) = 140618241825904
(id(name1) == id(name2)) = False


<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 115%; line-height: 135%;">

### 改用intern()函數：

In [12]:
import sys

name1 = sys.intern('Alex Van')
name2 = sys.intern('Alex Van')

print(f'{name1 = }\t\t\t{name2 = }')
print(f'{id(name1) = :<28}{id(name2) = }')
print(f'{(id(name1) == id(name2)) = }')

name1 = 'Alex Van'			name2 = 'Alex Van'
id(name1) = 140618241580080             id(name2) = 140618241580080
(id(name1) == id(name2)) = True


<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 115%; line-height: 135%;">

![](./assets/string%20interning-15.png)

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 115%; line-height: 135%;">

### 原規則：內含ASCII 128以上字元者不intern

In [13]:
os1 = 'Linux©'
os2 = 'Linux©'

print(f'{os1 = }\t\t\t{os2 = }')
print(f'{id(os1) = :<22}{id(os2) = }')
print(f'{(id(os1) == id(os2)) = }')

os1 = 'Linux©'			os2 = 'Linux©'
id(os1) = 140618241949744       id(os2) = 140618241959744
(id(os1) == id(os2)) = False


In [15]:
name1 = '張曼玉'
name2 = '張曼玉'

print(f'{name1 = }\t\t{name2 = }')
print(f'{id(name1) = :<20}{id(name2) = }')
print(f'{(id(name1) == id(name2)) = }')

name1 = '張曼玉'		name2 = '張曼玉'
id(name1) = 140618241984624     id(name2) = 140618241984336
(id(name1) == id(name2)) = False


<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 115%; line-height: 135%;">

### 改用intern()函數：

In [14]:
import sys

os1 = sys.intern('Linux©')
os2 = sys.intern('Linux©')

print(f'{os1 = }\t\t\t{os2 = }')
print(f'{id(os1) = :<22}{id(os2) = }')
print(f'{(id(os1) == id(os2)) = }')

os1 = 'Linux©'			os2 = 'Linux©'
id(os1) = 140618241958224       id(os2) = 140618241958224
(id(os1) == id(os2)) = True


In [16]:
import sys

name1 = sys.intern('張曼玉')
name2 = sys.intern('張曼玉')

print(f'{name1 = }\t\t{name2 = }')
print(f'{id(name1) = :<20}{id(name2) = }')
print(f'{(id(name1) == id(name2)) = }')

name1 = '張曼玉'		name2 = '張曼玉'
id(name1) = 140618241985296     id(name2) = 140618241985296
(id(name1) == id(name2)) = True


<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 115%; line-height: 135%;">

### 原規則：長度超過4,096個字元的字串不intern

In [18]:
str1 = 'A' * 4_097
str2 = 'A' * 4_097

print(f'{id(str1)=:<20}{id(str2)=:<20}')
print(f'{(id(str1) == id(str2)) = }')

id(str1)=94237236028400      id(str2)=94237236032560      
(id(str1) == id(str2)) = False


<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 115%; line-height: 135%;">

### 改用intern()函數：

In [19]:
from sys import intern

str1 = intern('A' * 4_097)
str2 = intern('A' * 4_097)

print(f'{id(str1)=:<20}{id(str2)=:<20}')
print(f'{(id(str1) == id(str2)) = }')

id(str1)=94237236045216      id(str2)=94237236045216      
(id(str1) == id(str2)) = True


<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 115%; line-height: 135%;">

### 原規則：字串由variable組合而成者不intern

In [21]:
first = 'first'
last = 'last'
name1 = first + 'last'
name2 = 'first' + 'last'

print(f'{name1 = }\t\t{name2 = }')
print(f'{id(name1) = :<20}{id(name2) = }')
print(f'{(id(name1) == id(name2)) = }')

name1 = 'firstlast'		name2 = 'firstlast'
id(name1) = 140618241618544     id(name2) = 140618241820272
(id(name1) == id(name2)) = False


<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 115%; line-height: 135%;">

### 改用intern()函數：

In [22]:
from sys import intern

first = 'first'
last = 'last'
name1 = intern(first + 'last')
name2 = intern('first' + 'last')

print(f'{name1 = }\t\t{name2 = }')
print(f'{id(name1) = :<20}{id(name2) = }')
print(f'{(id(name1) == id(name2)) = }')

name1 = 'firstlast'		name2 = 'firstlast'
id(name1) = 140618241820272     id(name2) = 140618241820272
(id(name1) == id(name2)) = True


<div style="font-family: 'jf open 粉圓 1.1', 'Noto Sans TC'; font-size: 115%; line-height: 135%;">



<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 115%; line-height: 135%;">

### 整數的interning

* 整數的內隱interning也有限制。
* 目前測試結果是256以內會自動intern，257以上則不會。

In [33]:
num1 = 256    # implicit的整數interning上限為256。
num2 = 256

print(f'{num1 = }\t\t{num2 = }')
print(f'{id(num1) = :<20}{id(num2) = }')
print(f'{(id(num1) == id(num2)) = }')

num1 = 256		num2 = 256
id(num1) = 140618291880144     id(num2) = 140618291880144
(id(num1) == id(num2)) = True


<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 115%; line-height: 135%;">



<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 115%; line-height: 135%;">

### 目前筆者不知道如何外顯intern整數

* 所以暫時無解。

In [32]:
from sys import intern

num1 = intern(257)    # implicit的整數interning上限為256。
num2 = intern(257)

print(f'{num1 = }\t\t{num2 = }')
print(f'{id(num1) = :<20}{id(num2) = }')
print(f'{(id(num1) == id(num2)) = }')

TypeError: intern() argument must be str, not int

<div style="font-family: 'Gen Jyuu Gothic Monospace Medium', 'Noto Sans TC'; font-size: 200%; line-height: 135%; text-align: center; color: Plum;">

<br><br>
# The End